**Notes**
---

I use k>m>n for nested iterators, i>j for dummy variables, and f()>g()>h() for explicit function/constraint maps, and I use random vector p~Uniform$[0,1)$ (Call:p=np.random.rand()). Due to the universality of the uniform, I theoretically could get by just using this but I also use (Call:p=np.random.rand()) for q~N(0,1), which can be transformed to q'~N($\mu,\sigma$) by q'=$\mu$+p*$\sigma$. Mutations of kp, 1/ti and 1/td will pull from a normal dist centered on its current value.

Parallelize:

-simulation (evaluation of fitness)

-

In [0]:
import numpy as np
from numpy import linalg
from numpy import random
import matplotlib as plt
import sqlite3


In [0]:
class Chromosome_db():
  def __init__(self):
    self.conn = sqlite3.connect('GeneticData.db')
    self.c = self.conn.cursor()
    self.create_table() # The class automatically sets up the table
  def create_table(self):
    self.c.execute('''DROP TABLE IF EXISTS chromosomes''')
    self.c.execute('''DROP TABLE IF EXISTS best''')
    self.c.execute('''CREATE TABLE chromosomes (name text, generation real, Kp float, Ti float, Td float, fitness float, diversity float)''')
    self.c.execute('''CREATE TABLE best (name text, generation real, Kp float, Ti float, Td float, fitness float, diversity float)''')
    #c.execute('''CREATE TABLE population (name text, pCrossover float, pMutation float, pFitness float, chromes float)''')
  
  def chromosome(self,chromo,generation):
    for i in range(len(chromo)):
      name,Kp,Ti,Td,fitness,diversity = chromo[i]
      self.c.execute('''INSERT INTO chromosomes VALUES (?, ?, ?, ?, ?, ?, ?)''', (name, generation, Kp, Ti, Td, fitness, diversity))
    #self.conn.commit()
  def best(self,chromosome):
    name,generation,Kp,Ti,Td,fitness,diversity = chromosome
    self.c.execute('''INSERT INTO best VALUES (?, ?, ?, ?, ?, ?, ?)''', (name, generation, Kp, Ti, Td, fitness, diversity))
    self.conn.commit()

  def print_chromosomes(self):
    self.c.execute("SELECT * FROM chromosomes")
    dbdata = self.c.fetchall()
    for i in dbdata:
      name,generation,Kp,Ti,Td,fitness,diversity = i
      print("Name: {}, Generation: {}, Kp: {}, Ti: {}, Td: {}, fitness: {}, diversity: {}".format(name,generation,Kp,Ti,Td,fitness,diversity))
  def print_best(self):
    self.c.execute("SELECT * FROM best")
    dbdata = self.c.fetchall()
    for i in dbdata:
      name,generation,Kp,Ti,Td,fitness,diversity = i
      print("Name: {}, Generation: {}, Kp: {}, Ti: {}, Td: {}, fitness: {}, diversity: {}".format(name,generation,Kp,Ti,Td,fitness,diversity))

  def get_chromosome(self,name):
    self.c.execute("SELECT * FROM chromosomes WHERE name = ?",(name,))
    return self.c.fetchall()[0]
  def get_best(self,name):
    self.c.execute("SELECT * FROM best WHERE name = ?",(name,))
    return self.c.fetchall()[0]
  def get_generation(self,generation):
    self.c.execute("SELECT * FROM chromosomes WHERE generation = ?",(generation,))
    return self.c.fetchall()
  
  def commit_db(self):
    self.conn.commit()
  def close_db(self):
    self.conn.close()

# Example of adding to database
chromo = Chromosome_db()
chromo.chromosome([("hello",.5,.5,.5,.7,.9)],0)
chromo.chromosome([("howdy",.6,.8,.3,.4,.8),("shalom",.2,.7,.8,.4,.4)],1)
chromo.print_chromosomes()
# Example of using the database functions
chromo.best(chromo.get_chromosome("howdy"))
print(chromo.get_best("howdy"))
chromo.print_best()
print(chromo.get_generation(0))


Name: hello, Generation: 0.0, Kp: 0.5, Ti: 0.5, Td: 0.5, fitness: 0.7, diversity: 0.9
Name: howdy, Generation: 1.0, Kp: 0.6, Ti: 0.8, Td: 0.3, fitness: 0.4, diversity: 0.8
Name: shalom, Generation: 1.0, Kp: 0.2, Ti: 0.7, Td: 0.8, fitness: 0.4, diversity: 0.4
('howdy', 1.0, 0.6, 0.8, 0.3, 0.4, 0.8)
Name: howdy, Generation: 1.0, Kp: 0.6, Ti: 0.8, Td: 0.3, fitness: 0.4, diversity: 0.8
[('hello', 0.0, 0.5, 0.5, 0.5, 0.7, 0.9)]


# New Section

In [0]:
class Sim:
  def __init__(self,td,t_end):
    self.td=td
    self.t_end=t_end
    length=int(np.ceil(t_end/td))
    self.x=np.zeros((2,length))
    self.u=np.zeros((1,length))
    self.y=np.zeros((1,length))
    self.e=np.zeros((1,length))
    

    #State Space Model
    self.A=np.array([[0.9045,0.006603],[-16.51,0.3763]])
    self.B=np.array([[3.82e-05],[0.006603]])
    self.C=np.array([[2500,0]])
    self.D=np.array([[0]])
    

  
  def simulate(self,chrome):
    #Calculating PID Controller Parameters
    K=chrome.Kp
    Ti=chrome.Ti
    Td=chrome.Td
    h=self.td
    n=100
    ad=Td/(n*h+Td)
    bd=n*K*ad
    Iold=0
    Dold=0
    r=1
    for m in range(len(self.x[0,:])):
      #PID Calculations for this run
      self.u[:,m]=r-self.y[:,m]
      P=K*self.u[:,m]
      I=Iold
      D=ad*Dold+bd*(self.u[:,m-1]-self.u[:,m])
      c=P+I+D
      unstable=0
      #State Transition Calculations
      if m < int(np.ceil(self.t_end/self.td))-1: #Don't execute on last run
        self.x[:,m+1]=np.dot(self.A,self.x[:,m])+(self.B*c).reshape(2)
      self.y[:,m]=np.dot(self.C,self.x[:,m])+(self.D*c).reshape(1)
      if self.y[:,m]>1e6:
        print('Crossed at m=',m)
        unstable=1;
        break
      self.e[:,m]=np.abs(r-self.y[:,m-1])*m*h+self.e[:,m-1]
      
      #Redefine for next iter
      Iold=Iold+K*h/Ti*self.u[:,m]
      Dold=D
      
    #ITAE
    #cost=np.dot(self.e,ones(len(self.e[0,:])))
    if unstable==1:
      cost=1e6
    else:
      cost=self.e[:,-1]
      
#     print('y='+str(self.y))
    return cost

In [0]:
class Chromosome:
  def __init__(self,name,data):
    self.name = name
    #self.generation = 0
    self.data=data
    self.Kp = data[0]
    self.Ti = data[1]
    self.Td = data[2]
    self.fitness = 0
    self.diversity = 0
    self.aggfitness = 0
    
  def cFitness(self):
    #function of error. call upon Sim. ITAE?
    self.fitness = 0
    self.sim = Sim(0.01,10)
    self.fitness = -np.log(self.sim.simulate(self))+13.82
    
  def aggFitness(self):
    self.aggfitness = self.fitness+self.diversity
    
  def update_chromosome(self, chromo):
    self.name,self.Kp,self.Ti,self.Td,self.fitness,self.diversity = chromo
    
  def output_chromosome(self):
    return (self.name,self.Kp,self.Ti,self.Td,self.fitness,self.diversity)
  
  def updateName(self, name):
    self.name = name
    

class Population:
  """
  pCrossover=probability of a crossover event
    Crossover event is defined by a swap of a data point with its respective data point in another chromosome.
    e.g.: A crossover event between [1,2,4] and [2,2,2] 
  pMutation=probability of a mutation event
    All mutations are pointwise mutations that are bounded rerolls of the data point with expectation equal to the original point.
  
  pFitness=Rank 0 fitness of the population=probability that the most fit individual will reproduce
  
  chromes=list of chromosomes in the population
  
  count=number of chromosomes in population (convenience)
  
  mean=centroid of pop
  """
  
  def __init__(self,name,pCross,pMut,pFit,gen,chromes):
    assert pCross<=1 and pCross>=0 and pMut<=1 and pMut>=0
    self.name=name
    self.pCrossover = pCross
    self.pMutation = pMut
    self.pFitness = pFit
    self.chromes = chromes.copy()
    self.count = len(chromes)
    self.mean = np.zeros(len(self.chromes[0].data))
    self.gen=gen
  
  def cDiversity(self):
    #This calculates the cardinal diversity (float) of each chromosome by distance from centroid
    #Diversity needs to be less important for future generations (SA)
    for k in self.chromes: #This could probably be vectorized by turning the population into a matrix of chromosome columns
      self.mean += k.data/self.count
    for k in self.chromes: #Needs to be done after mean
      diff=k.data-self.mean
      k.diversity=np.dot(diff,diff)*np.exp(-self.gen/20) #sum of entries squared times a transient
  
  def sortPop(self):
    #Sort Population.chromes by fitness score + diversity score high to low
    #Can do arbitrary tie-breakers or aggregate probability range and inherit rank from index
    self.chromes = sorted(self.chromes, key = lambda c: c.aggfitness, reverse = True)
    
  def oFitness(self):
    #ordinal fitness definition goes here
    #rank-->probability
    #Linear combination of objective (cFitness) and diversity (cDiversity)
    
    self.sortPop()
    
    
  def getChrome(self, index):
    return self.chromes[index]
    
  def propagate(self):
    newPop=[]
    local = self.chromes.copy()
    for k in range(self.count):
      #Geometric Random Event Generator:
      #Wikipedia/Geometric Distribution/Related Distributions Bullet Point 4
      #If I transform u.r.v. p-->n by this formula, this will be geometrically distributed with probability parameter self.pFit
      p=np.random.rand()
      n=np.log(p)/np.log(1-self.pFitness)
      n=int(np.floor(n))
      #This has an equivalent distribution to a system with N=self.count+1 events where the Nth event is a resample event from the distribution
      #The mechanics of this wrapping is inhereted from the memoryless property of the geometric distribution
      n=n%(self.count)
      local = self.chromes[n].data.copy()
      xnew=Chromosome("x"+str(k),local)
      newPop.append(xnew)
      #print(k)

    return newPop
  
  
  def resetNames(self):
    for num,k in enumerate(self.chromes):
      k.name = 'x' + str(num)
      
  def rCrossover(self):
    #Note: Because our chromosomes only carry 3 information values,
    #All possible correspondant swaps between 2 chromosomes can be expressed by a swap of only 1 value
    #Maybe we could consider 3-way swaps because there are 3 degrees of freedom in this tuning problem, so this might explore the space better.
    for k in range(self.count):
      for m in range(self.count):
        #Prevent 3 Cases:
        #1.) Swap Between the same instance of a chromosomes
        #2.) Swap between chromosomes with the same data
        #3.) 2 Swaps in one call of self.rCrossover()
        if k>m and (self.chromes[k].data!=self.chromes[k].data).any():
          #Uniform Random Event Generator
          p=np.random.rand()
          i=p*3//1
          i=int(i)
          #Juggler for the swap between two chromosomes
          j=[self.chromes[k].data[i],self.chromes[k].data[i]]
          m.data[i]=j[0]
          k.data[i]=j[1]
  
  def rMutate(self):
    sd=1
    for k in self.chromes:
      #Point Mutation Event
      #Fundamental Bridge, Probability of an event is equal to the expected value of its indicator
      #p[0] indicates whether or not to mutate
      #p[1] indicates what mutation event will occur
      #Note: Due to mutual exclusion, the mutation events are not independent
      p=np.random.rand(2)
      if p[0]<=self.pMutation:
        #I can either make this mutate frequently with low variance or infrequently with high variance.
        #Uniform Random Event Generator
        #Case Structure:
        #0:Mutate Kp
        #1:Mutate Ti
        #2:Mutate Td
        #3:Mutate All not Kp
        #4:Mutate All not 1/Ti
        #5:Mutate All not Td
        #6:Mutate All
        i = 7*p[1]//1
        i = int(i)
        #Try and think of a smarter way to do this.
        #I could also make the mutations have their percent change in size be normally distributed about 1...
        if i in range(3):           k.data[i] += np.random.randn()*sd           #Cases 0,1,2
        if i in range(3,6):               
          for m in range(3): 
            if i!=m+3:              k.data[m] += np.random.randn()*sd           #Cases 3,4,5
        if i==6:
          for m in range(3):        k.data[m] += np.random.randn()*sd           #Case 6
        
        #Ensure Coeffs don't generate positive feedback (will integrate ref signal-->unstable)
        #Recall that the data points represent kp, 1/ti, and 1/td, so there aren't domain failures generated from this random mutation.
        for m in range(3): k.data[m] =  np.max([k.data[m],1e-3]) 
      
  
      
      
      
        
      

In [0]:
#Initial Chromosome Population
#Random init of 

N = np.arange(30)
x = list()
for k in N:
  x.append(Chromosome("x"+str(k),np.random.rand(3)*10))

  
  
for i in range(10):
  for k in N:
      x[k].cFitness()
    #print(x[k].name)

  #Determine Population Statistics
  parentPop=Population("Parent",1,1,0.2,0,x)
  parentPop.cDiversity()

  for k in N:
    parentPop.chromes[k].aggFitness()

  parentPop.oFitness()

  #Generate Offspring Population
  childPop=Population("Child",parentPop.pCrossover,parentPop.pMutation,parentPop.pFitness,1,parentPop.propagate())

  #reset names
  childPop.resetNames()

  #Crossover Offspring Population
  childPop.rCrossover()

  #Mutate Offspring Population
  childPop.rMutate()

  # Debug

  #print("Testing Chromosome Object")
  for k in parentPop.chromes:
    print("Data for "+k.name+":")
    print("Coefficients:\t",k.data)
    print("Fitness:\t",k.fitness)
    print("Diversity:\t",k.diversity)
    print("Fit+Div:\t",k.aggfitness,"\n")
  print('='*50)
  print('='*50)
  print('='*50)
  
  x = childPop.chromes
  
#   for k in childPop.chromes:
#     print("Data for "+k.name+":")
#     print("Coefficients:\t",k.data)
#     print("Fitness:\t",k.fitness)
#     print("Diversity:\t",k.diversity)
#     print("Fit+Div:\t",k.aggfitness,"\n")
  # print("="*20)
  # print("Testing Population Object")
  # print("pCrossover=",parentPop.pCrossover)
  # print("pMutation=",parentPop.pMutation)
  # print("pFitness=",parentPop.pFitness)
  # print("Parent Pop=")
  # for k in range(parentPop.count): 
  #   print("\t",parentPop.chromes[k].name)#long list
  # print("Child Pop=")
  # for k in range(childPop.count): 
  #   print("\t",childPop.chromes[k].name)#long list
  # print("count=",parentPop.count)
  # print("mean=",parentPop.mean)

Data for x25:
Coefficients:	 [0.8942547  9.90249694 0.99220374]
Fitness:	 [6.00956234]
Diversity:	 66.48032703167335
Fit+Div:	 [72.48988938] 

Data for x12:
Coefficients:	 [9.47315937 7.68622861 0.24178369]
Fitness:	 [2.49278919]
Diversity:	 59.770340916592936
Fit+Div:	 [62.26313011] 

Data for x9:
Coefficients:	 [1.2321168  9.43472088 8.64592367]
Fitness:	 [5.18888431]
Diversity:	 49.36028715460455
Fit+Div:	 [54.54917146] 

Data for x20:
Coefficients:	 [9.08022234 0.5195755  9.92731128]
Fitness:	 [0.48457344]
Diversity:	 52.55996120178564
Fit+Div:	 [53.04453465] 

Data for x24:
Coefficients:	 [2.53048918 0.12571805 1.02088251]
Fitness:	 [0.40079356]
Diversity:	 42.584280176221554
Fit+Div:	 [42.98507374] 

Data for x2:
Coefficients:	 [9.62066334 0.26010528 4.78812981]
Fitness:	 [-0.23107659]
Diversity:	 39.8084623559529
Fit+Div:	 [39.57738577] 

Data for x27:
Coefficients:	 [0.89564685 5.87540139 9.32939949]
Fitness:	 [5.37872517]
Diversity:	 33.070793467927764
Fit+Div:	 [38.44951864] 

In [0]:
#prototyping block
z0=np.array([[1,2,3],[0,0,0],[0,0,0]])
z1=np.array([1,3,6])
print(z0[1])


[0 0 0]


In [0]:
"""
Sorter to sort X objects by fitness
We could have sorted in place to make it simpler,
but if we decide to make a class for the array of chromosomes
then we can add this there.


Sort takes in the Chromosome iterable and sorts the objects based on fitness
ascending = lowest to highest
descending = highest to lowest
"""

def sortX(iterX, descending = False):
  rank = sorted(iterX, key = lambda c: c.fitness, reverse = descending)
  return rank 


In [0]:

"""
10.3.101.176
mpiprojects


This is the main execution code for mpi
It is broken down into 3 steps. Steps 2 and 3 are part of For loop

1. Initialization on rank 0 (master)
..... Typecasting as numpy array
..... Splitting chromosome np array into shape(size, len (+- remainder) /size)

[BEGIN FOR LOOP]
2. Simulation
..... Scatter np array to slave nodes
..... Slave nodes perform cFitness (SIMULATION)
..... Slave nodes gather numpy array

3. Genetic progression Master
..... Numpy array is reshaped back to vector (FX)
..... fx added to parent population
..... diversity calculated
..... aggfitness calculated
..... ofitness calculated
..... child population generated
..... reset names
..... crossover
..... mutation
..... x is reassigned to childpop chromes
..... repeated
[END FOR LOOP]
"""

# if rank == 0:

    # Create initial chromosomes / population
x = [Chromosome("x"+str(k), np.random.rand(3)*10) for k in range(30)]

    # open file for writing

#     f = open("output.txt", "a")


# Master Loop
for i in range(1000):

        # Cast as numpy array
        x = np.array(x)



            # Simulation Code

        for k in x:
            k.cFitness()

        #Database Call

        parentPop = Population("Parent", 1, 1, 0.2, 0, x)
        parentPop.cDiversity()

        for k in parentPop.chromes:
            k.aggFitness()

        parentPop.oFitness()

        # Generate Offspring Population
        childPop = Population("Child", parentPop.pCrossover, parentPop.pMutation,
                              parentPop.pFitness, 1, parentPop.propagate())

        # reset names
        childPop.resetNames()

        # Crossover Offspring Population
        childPop.rCrossover()

        # Mutate Offspring Population
        childPop.rMutate()

        # Print

        print('--{}--'.format(i))
        print("Data for "+parentPop.chromes[0].name+":")
        print("Coefficients:\t", parentPop.chromes[0].data)
        print("Fitness:\t", parentPop.chromes[0].fitness)
        print("Diversity:\t", parentPop.chromes[0].diversity)
        print("Fit+Div:\t", parentPop.chromes[0].aggfitness, "\n")



        # Update x to be child props chromosomes
        x = childPop.chromes

print('completed')


--0--
Data for x25:
Coefficients:	 [9.52032469 9.51615958 9.6924176 ]
Fitness:	 [2.57366292]
Diversity:	 57.761165511378124
Fit+Div:	 [60.33482843] 

--1--
Data for x28:
Coefficients:	 [0.29852705 5.13149782 0.1296545 ]
Fitness:	 [4.12098435]
Diversity:	 76.21093698669225
Fit+Div:	 [80.33192134] 

--2--
Data for x11:
Coefficients:	 [ 9.11167435  9.58024296 10.67139518]
Fitness:	 [2.61825434]
Diversity:	 121.65369035681141
Fit+Div:	 [124.2719447] 

--3--
Data for x25:
Coefficients:	 [ 8.11383005 10.6485157   0.32443914]
Fitness:	 [2.72293178]
Diversity:	 94.43093718152141
Fit+Div:	 [97.15386896] 

--4--
Data for x5:
Coefficients:	 [ 0.80224198  0.2835181  10.21523299]
Fitness:	 [3.59197772]
Diversity:	 182.90195185048395
Fit+Div:	 [186.49392957] 

--5--
Data for x24:
Coefficients:	 [10.0199332  10.29086254  0.14975132]
Fitness:	 [2.56541114]
Diversity:	 147.00696519349805
Fit+Div:	 [149.57237634] 

--6--
Data for x1:
Coefficients:	 [ 0.80224198  0.2835181  11.11760234]
Fitness:	 [2.3836

In [0]:
import matplotlib.pyplot as plt


data = [(1,1), (2,2), (3,3)]

fig, ax = plt.subplots()

x_val = range(13)
y_val_wdb = [31.866124868392944, 25.07438588142395, 17.10246253013611, 22.855093955993652, 17.73931312561035, 15.269845008850098, 22.304879188537598, 17.568716049194336, 16.951478481292725, 17.972474813461304, 15.877592086791992, 14.662986993789673, 11.805564880371094]
y_val_wdbb = [45.15414619445801, 166.09449315071106, 65.17989253997803, 21.273635625839233, 21.693725109100342, 60.70862174034119, 96.83690571784973, 21.22640895843506, 23.73573637008667, 26.4404137134552, 23.43152356147766, 128.50756573677063, 55.56160855293274]
y_val_wodb = [13.031464576721191, 7.787649631500244, 5.341392517089844, 4.428461313247681, 3.330021858215332, 2.897095203399658, 2.637894868850708, 2.6456315517425537, 2.368424415588379, 1.9740374088287354, 1.8654940128326416, 1.7016429901123047, 1.5876288414001465]
plt.title('Runtime with DataBase IO')
plt.xlabel('Nodes')
plt.ylabel('Seconds')
ax.plot(x_val,y_val_wdb)
ax.plot(x_val,y_val_wdb,'xk')
plt.savefig('./Runtime_Without_Database.png')
plt.show()

In [0]:
fig, ax = plt.subplots()
plt.title('Runtime with DataBase Frequent Commit')
plt.xlabel('Nodes')
plt.ylabel('Seconds')

ax.plot(x_val,y_val_wdbb, 'r')
ax.plot(x_val,y_val_wdbb, 'xk')
plt.show()


In [0]:
fig, ax = plt.subplots()
plt.title('Runtime without Database IO')
plt.xlabel('Nodes')
plt.ylabel('Seconds')
ax.plot(x_val,y_val_wodb, 'g')
ax.plot(x_val,y_val_wodb,'xk')
plt.savefig('./Runtime_With_Database.png')

plt.show()



In [0]:
fitness =[4.841290278869346, 4.841290278869346, 5.236217077076043, 4.486631717767011, 5.238299328576547, -1.1606654148773714, 5.238015445223892, 2.6729049590341774, 5.2428835210159495, 2.4922677782067275, 5.2417671657841005, 2.4844852260553036, 2.4482346911300574, 5.2848053718659855, 2.3495547451268273, 5.284438183073002, -1.6405827277585665, 5.2847061188829745, 5.298164279344766, -1.7619306053012043, 5.291862183742321, 4.526047646313604, 4.43354946710221, 4.736983123466564, 4.932643248266606, 3.591687095652686, 5.016064980747409, 3.532287803700422, 5.073798968899521, 3.4914301680630047, 5.105337008013825, -2.163204137098525, 5.125092294189381, 3.4970687002163796, 5.1643970298278585, 4.717207331565202, 5.166332368673997, 4.8012503842989425, 4.807574752136167, 5.171833515094775, 4.792005566952422, 5.19628161780224, 4.5328010332037465, 4.5269691034271204, 5.229248656072311, 4.339805059423002, 4.681851622787516, 3.96454938979252, 3.9375158546129523, 4.57891308447536, 4.229791682108658, 4.790786988523363, 3.945849834249314, 4.518097174082074, 4.604671344494545, -2.6497551272235498, 4.600029568647729, -2.6833110742895165, 4.61257574194064, 0.12808155947736388, 4.698252170562485, 4.0921184778420905, 4.708267964166373, 4.078141880109589, 4.764842083432891, -2.7777142137609054, 4.808874268802828, -2.807865896220193, 4.824435923333471, 3.4583694907653353, 4.8475413299929055, 4.860525659945187, -2.8523647135504184, 2.9805827264325426, 4.87966805743938, 4.009980070028437, 4.059525396571198, 4.920737881443175, 4.900430092425459, 3.85782633908838, 4.9278502309830845, 3.1983046332360114, 4.924172383801084, 4.930995010084599, 3.9566958294469075, 3.633363747632954, 4.946880851005762, 3.5096298670460406, 3.521663207610203, 4.964646022448443, -2.98293239419273, 4.990341450430893, 4.993285504663499, 3.023538465615717, -2.9906971331116523, -2.9940013318832754, 5.014670920862102, 3.8540972920353376, 5.0314312444495055, 3.913061323821518, 5.010841681832456, -3.0218321535546764, 5.075620377519467, 5.017101855808994, 5.065656368077091, 5.0203638680988885, 5.0725656768295355, 5.006493769321953, 5.063932070817936, 4.991176435572374, 5.0539363124584185, 5.01071560313545, 5.05192415047436, 5.003280832747082, 5.037179448837591, 5.003716259203408, 5.055954385315532, 5.0750164031950415, 4.995669193216283, 4.992639109851183, 5.082033368308805, 4.989124882745692, 5.076981698925282, 4.964273998551528, 4.952819066717067, 5.079258745812195, 5.07940091475534, 4.939417371261394, 5.083615111210886, 4.93945267831573, 4.944038294031685, 4.926162986008931, 4.946055333599624, 4.902698438356454, 4.940146628659006, 4.916027699762074, 4.936470637932986, 4.928827106317472, 4.954035469805742, 4.931125554045279, 4.950633047390179, 4.934722585209597, 4.942389140558602, 4.929531557259516, 4.930893077756938, 4.945882727306092, 4.927430197243863, 4.947694979291461, 4.9548598675607405, 4.9491303164741485, 4.9887752356915165, 4.974811677336396, 4.95865035737028, 4.955777330568354, 5.008862710775414, 4.94344133597097, 5.003311531626315, 4.999908804241631, 4.953599203055996, 4.999739895103314, 4.956569521884063, 4.95341411175956, 5.02613526924406, 5.008949044215495, 4.957678891797629, 4.957743159046478, 5.0034484704398015, 4.949063591450411, 5.018133809846821, 4.9335601218748995, 5.008372025399387, 4.927293722827418, 5.008630848950681, 5.004905509132682, 4.922634993688481, 5.014130317922659, 4.930616228676644, 5.014341227569389, 5.039124584370125, 4.91632896712065, 5.0203231176911665, 4.905312963927049, 4.935434773981912, 4.92493024566085, 4.934527927325963, 4.879829170313064, 4.926276196824251, 4.949833902645079, 4.9022463731944566, 4.976703795446811, 4.899543639036322, 4.85507171253788, 4.992167570244495, 4.991505860064578, 4.881084505615073, 5.007891753181632, 4.849050486765831, 5.029893063404078, 4.830745044261773, 4.831146474193487, 5.065603723792092, 5.065477933888319, 4.810809194765989, 5.077349019867594, 4.860906541610769, 5.080902262262011, 5.098579803474388, 4.810424504023731, 5.113544212136386, 4.804618297743081, 5.107623073716162, 5.108269650916959, 5.10858358830477, 5.114742538469995, 5.115903536118509, 5.129848639440448, 5.137001127149016, 5.070621281080788, 5.167779138254032, 5.068290069782883, 5.173047804828133, 5.0558672916146, 5.181446907979215, 5.176264031129451, 5.1773012193950905, 5.1610111776911225, 5.212347074171692, 5.144251503596026, 5.219220499147591, 5.147874272207961, 5.225430001603247, 5.11701710378312, 5.249458712343955, 5.089589454807193, 5.2421743640519125, 5.249877662330306, 5.068845414502096, 5.129179182871663, 5.12872818651322, 5.069613477132007, 5.1422858424717575, 5.078628141332652, 5.155364621386489, 5.086478460324951, 5.176885114288645, 5.0838767855082505, 5.079822502835098, 5.200021116917075, 5.079284986237916, 5.216769470289947, 5.066486238669688, 5.064063314647422, 5.225284173952, 5.0563021621875475, 5.241793097291303, 5.0425499627320605, 5.224733167947903, 5.214279139762464, 5.210460997473817, 5.050419436511735, 5.05769704910535, 5.233643072769544, 5.244577859405503, 5.062191170407544, 5.263640920374476, 5.066643266798165, 5.2788799674978755, 5.279027137645857, 5.05717070554622, 5.274957344524573, 5.057441535352082, 5.303665309450231, 5.058126662750473, 5.044997421838488, 5.058460524502555, 5.050783810064505, 5.051300757610699, 5.051773316206303, 5.070195067315682, 5.034468055073155, 5.034474463996977, 5.069696321555906, 5.034675891859738, 5.064352237105661, 5.059697054475201, 5.060767599297828, 5.067960537968581, 5.060897584839173, 5.0701020153600265, 5.0642510559926635, 5.0621488184855234, 4.991786270562141, 4.992105074404588, 5.03440430322096, 4.994651239930125, 5.085238991895428, 5.095532272388548, 5.072682066124864, 5.073771993884661, 5.095053637510334, 5.067225059864953, 5.101771476011871, 5.06646643669967, 5.123818203232444, 5.064126192956364, 5.154919379761363, 5.062007272605307, 5.1637907883535, 5.051014619358757, 5.179920693507858, 5.06191023726508, 5.061723829497531, 5.068026757867761, 5.060595000404634, 5.208499319571587, 5.208527093124545, 5.048647444456343, 5.210635920654319, 5.035993139322919, 5.222930016644655, 5.032733685096034, 5.226643080047733, 5.011164618320974, 5.011525560716903, 5.244420122651864, 5.00103331539775, 5.265047314413218, 4.991587976030942, 4.984082283105934, 5.2867535507628, 5.286660816628505, 4.978016738370931, 4.9757837187014715, 5.285448593042188, 4.987796188545769, 5.294221948613735, 4.985727579551217, 5.303798406586536, 4.981044115241515, 4.975159347815305, 5.300722669323234, 4.973794815699067, 5.301340298256868, 4.967625613281172, 4.9794722948024095, 4.968251316973507, 4.969242683444726, 4.968672755499011, 4.968764352826842, 4.9694041144079595, 4.99376512915965, 4.970624231360786, 5.0030412435113085, 4.989435785324247, 4.956931572778251, 4.95763544643447, 5.027947791690433, 5.028530037697497, 4.947770133118485, 5.041382452986742, 4.92751875977919, 5.039559428983006, 4.9408896554680855, 5.046355508465988, 4.888927102722404, 5.057159767572713, 5.073538901574597, 4.8604704325396035, 5.07150068876412, 4.837074299024799, 5.065746396541819, 4.851922004859059, 5.090006859835851, 4.850074576960322, 5.080292126683679, 4.832799028333266, 5.085355712446466, 4.830980512785045, 4.828418211057109, 5.083599524347548, 5.0866302719247045, 5.090226511066119, 4.803411496993629, 5.107911124776466, 4.789837418179658, 5.1140279440973195, 4.777145423541102, 5.108223951790231, 4.770360794041757, 5.136965076817113, 5.152412402200978, 4.754210438003824, 4.754345347447183, 5.1544881673224, 4.74474468658828, 5.205254187850846, 5.204776543355781, 4.721992975843666, 5.216890400858388, 4.703026713453482, 5.207531551297288, 4.692660818948102, 5.217981336504193, 5.220848717845124, 4.682000007127357, 4.682982974547894, 5.255266343291632, 4.668368792951121, 5.264012777820435, 5.254204084344378, 4.671211318587737, 5.259656371362384, 4.67991656826368, 5.276810226814829, 4.657905144970162, 4.651688260825814, 5.289252155344542, 4.638650953001864, 5.283149652444216, 4.625867337295114, 5.298450608457729, 4.61731754296688, 4.605559609148408, 5.297464362204186, 4.603805754827027, 5.297244848670109, 5.2972108132702385, 4.586726236070708, 5.297134539967347, 5.297509410391003, 4.599063406747705, 5.2937230506074755, 4.594025528698262, 5.296226995791537, 4.579505360677068, 5.301361950306438, 4.572542165009404, 4.559050398557586, 5.303794876286398, 5.303794895560641, 4.522959863702818, 4.528391517406252, 5.303795702159812, 5.292851953285341, 4.495295852822629, 5.303795607760996, 4.4705759356715085, 5.303794036821889, 4.439743097782888, 4.4836241396878656, 4.433657656364776, 4.495009598371148, 4.445622995280157, 4.49184090123415, 4.462044044855325, 4.502637794908781, 4.47582135246029, 4.499284646221236, 4.441737552603941, 4.442423922237349, 4.514251484685243, 4.463782030721127, 4.511269412650547, 4.492729071174772, 4.509398616401187, 4.496517647395596, 4.498734139614662, 4.498752874634803, 4.494651276632169, 4.504910309707347, 4.503367875174423, 4.500511738186637, 4.508111259671923, 4.511800097290688, 4.498353429655845, 4.502074930675798, 4.482469035506776, 4.503971399217669, 4.482572064084259, 4.513149437829705, 4.487967849066166, 4.507452332944169, 4.49396159864375, 4.517169250196671, 4.518055157948703, 4.497177969298983, 4.531337274912991, 4.489751717729213, 4.486281263335339, 4.489512251655874, 4.487194742152324, 4.485927296891676, 4.495531865295966, 4.5032943433014765, 4.459877411394693, 4.493344492934947, 4.4986693907163176, 4.488168430758218, 4.498261916203104, 4.471282021021278, 4.509564772298248, 4.499992738197728, 4.471192320048912, 4.495872023210955, 4.470355539639179, 4.513007443770935, 4.510221990445643, 4.517714468004854, 4.463345514676584, 4.50969304195649, 4.4687819654884695, 4.5179576161625725, 4.518648647184049, 4.418784839181319, 4.524291131893605, 4.523870769090399, 4.413304314744407, 4.5177202356324475, 4.395078567512117, 4.398601707657624, 4.407403326319624, 4.536801528859778, 4.523192423128531, 4.379070724680885, 4.367035394115804, 4.535096469229742, 4.543026056020317, 4.3775747220552095, 4.551270977402803, 4.362534360532603, 4.541647255040809, 4.376383431416178, 4.377560297486363, 4.537065583904823, 4.368116721610999, 4.529065299865204, 4.534667927199987, 4.362717845609646, 4.3675337572627075, 4.5394571928553, 4.388982170843979, 4.538123273894717, 4.53760552802169, 4.340911687656961, 4.553911869010323, 4.338741216712469, 4.558605449795918, 4.54722289959321, 4.324114402206298, 4.5512912651639414, 4.321827549549614, 4.5539340412021705, 4.555808193973174, 4.564493374833276, 4.533861336302866, 4.583013579672427, 4.535656733314399, 4.545077463105944, 4.583867384789439, 4.53612337069219, 4.593277231731051, 4.600301543640004, 4.524475939065482, 4.595403964559269, 4.513839885551883, 4.510245445719665, 4.604807152998244, 4.607222410400592, 4.507790802769252, 4.507907305107203, 4.618881796683928, 4.501836315103432, 4.624582993786193, 4.624065365910685, 4.505667930168789, 4.501284704673667, 4.656637112215249, 4.498330395922617, 4.659909780897925, 4.504770836666744, 4.67093925224842, 4.666912757344649, 4.6654618402020915, 4.671385319796798, 4.657192314728208, 4.678871138412726, 4.679797511067578, 4.654272575372174, 4.685627901965518, 4.6500025437950825, 4.674138972792424, 4.647439139847213, 4.681015205385236, 4.645436373615187, 4.688465396127317, 4.643878769418617, 4.686440226664827, 4.631375228664556, 4.683600590342669, 4.629566533665528, 4.627897049724268, 4.634406181601522, 4.662988957764096, 4.630784383962373, 4.625197239865276, 4.688632325310087, 4.626374887543422, 4.7025371921826356, 4.617939157765054, 4.715888790354585, 4.609319778989544, 4.725832575841208, 4.725056760736718, 4.5969254420381205, 4.731153136079749, 4.580447772106414, 4.73130448974392, 4.7288072122975215, 4.582947072037497, 4.724354866228124, 4.726266241916882, 4.727981411537092, 4.616123086308564, 4.609660570967398, 4.7535736012000775, 4.60199956904712, 4.748414344010044, 4.748859040176496, 4.606919336433712, 4.745274650442584, 4.609555957271345, 4.617292490572172, 4.754446542195289, 4.617902245291036, 4.7713576302706215, 4.764505667500231, 4.61183645162143, 4.757544637875792, 4.602817072463278, 4.769295118544694, 4.590296234544873, 4.594741935948051, 4.782904370647323, 4.582143732159398, 4.795834738533161, 4.793521090108433, 4.574957545914577, 4.578539303526844, 4.7981553577715275, 4.7969787977228595, 4.578796408598215, 4.808269235744916, 4.590525749037667, 4.598117068660816, 4.572979288959216, 4.6071232241396824, 4.572690465295954, 4.612781968381073, 4.623045690607443, 4.549568010382403, 4.625908031740552, 4.537724139895184, 4.630442978433896, 4.536813199794032, 4.648882286867343, 4.535122195905467, 4.65675927117406, 4.532162435206239, 4.6700541561009, 4.669757154929638, 4.6745676586685025, 4.666360620140807, 4.6868339923582365, 4.649077095184037, 4.693759292152951, 4.693027980497414, 4.642083874035844, 4.694214330105112, 4.620687161256951, 4.717826176539264, 4.620620849487608, 4.621150947060187, 4.730564072284816, 4.619686184891627, 4.743860555990487, 4.612749641962628, 4.743674054928464, 4.744087475883568, 4.745698318421592, 4.745390580419096, 4.736862041044981, 4.744649744722816, 4.7561138618911425, 4.744653525956869, 4.758859312591223, 4.747934744736451, 4.746965018008964, 4.758611206535434, 4.758879058930161, 4.777971757664995, 4.761374718991055, 4.781523625717684, 4.784630952273888, 4.768847942851897, 4.765350401068808, 4.802318285913271, 4.764530495135297, 4.8065969534626145, 4.765123718229656, 4.807001528205985, 4.75997825289995, 4.809238099406533, 4.807129335542827, 4.765642452933461, 4.812415486736656, 4.767433877147306, 4.794762212763619, 4.794346457615285, 4.756627522302615, 4.757378113678188, 4.8242453360109785, 4.823887225594579, 4.760533034778877, 4.827617792917382, 4.818966261753394, 4.795885261968628, 4.8369546628832065, 4.796691606724661, 4.795853180031289, 4.858990470697725, 4.797179278710095, 4.797241365337513, 4.797538949285734, 4.863266763589776, 4.868338991602053, 4.806871654107875, 4.873575516136601, 4.804694082220218, 4.798629316378669, 4.878007774851859, 4.78345873568834, 4.876604569255177, 4.77606684237251, 4.897972768932819, 4.783853602997157, 4.784486862328331, 4.881184332128257, 4.775130459550349, 4.879402971669494, 4.773593827780136, 4.8842672618689935, 4.763708196810596, 4.893382333844965, 4.762788324916318, 4.760988324547226, 4.757973533176399, 4.7677324371408165, 4.771997626778239, 4.789183236168968, 4.801834403205342, 4.805655326144802, 4.806619923187888, 4.810214964163126, 4.81879065549397, 4.814583538787939, 4.823834699143832, 4.843287200428506, 4.81035073347031, 4.859882100724361, 4.800392636663849, 4.8812671679922115, 4.797248920083611, 4.888833133033652, 4.807596257255197, 4.8884964327424605, 4.9028020788108435, 4.902989702524822, 4.926944014535842, 4.940092460506339, 4.950297194939866, 4.958818702656645, 4.96515243571864, 4.97454834542145, 4.975393689192472, 4.984973179419823, 4.979634948703442, 4.985113186970249, 4.992274502114482, 4.997015404985779, 4.999477108720635, 5.005894439623592, 4.998685638948684, 4.994382061910613, 5.021603959089063, 4.99335543740556, 5.002438441767435, 5.038562964375576, 5.006047761506048, 5.037161672696859, 5.024490484560632, 5.024540587450417, 5.040888886306021, 5.03473936711722, 5.045941488762839, 5.034118807581956, 5.0459883678503505, 5.03611606208662, 5.038878815770602, 5.050262740666076, 5.038817138585287, 5.0745151786645515, 5.074854746759005, 5.034764505602597, 5.077500485515943, 5.051625304834678, 5.085951010284965, 5.070474766673195, 5.092330336085814, 5.067665144335157, 5.092241113382574, 5.072833639138905, 5.088643980527275, 5.097680994652963, 5.0853928116505, 5.103750159925845, 5.077980848580163, 5.13341845639143, 5.084946113663948, 5.172871961505354, 5.08217488389076, 5.085100005445733, 5.207021261241055, 5.076113361047936, 5.211920009346899, 5.075214458149963, 5.215745243302543, 5.076882299346867, 5.2135099069708914, 5.077822671867795, 5.226085041057248, 5.058371177134518, 5.2375362589340035, 5.068055890636785, 5.238093580681268, 5.058750887478929, 5.267367931310336, 5.064102524734636, 5.265769814417197, 5.059660396386398, 5.0573340897099275, 5.303802916079864, 5.057051060197521, 5.299427146362799, 5.056672371015633, 5.303802925128405, 5.064722060066284, 5.0679122673398, 5.30048289206985, 5.303803139445481, 5.0764350707716, 5.303803141256944, 5.303803742406693, 5.303803145674605, 5.30377738294858, 5.303802495394571, 5.303804449157642, 5.303802281702795, 5.303804421018993, 5.303802117388916, 5.301214968384013, 5.29585227813199, 5.298935118227337, 5.2971625213798355, 5.303802208339416, 5.297282025714988, 5.302356600651807, 5.293146230459842, 5.282513492212436, 5.303802016856508, 5.286275376905811, 5.303802021713652, 5.2862414582501795, 5.300942958311479, 5.294424236207357, 5.303801637310393, 5.303801730973987, 5.3038021831692586, 5.30380174845363, 5.303802010589154, 5.303801689431232, 5.30380197725856, 5.303801793157534, 5.3038019899767015, 5.3038017012119845, 5.30380198724697, 5.303096329487744, 5.303801705400568, 5.303801599176454, 5.3038017993835265, 5.303801733708864, 5.303304982698272, 5.303801522614092, 5.298355563575209, 5.30380200092976, 5.3017767245481195, 5.303328262232977, 5.30027172303927, 5.303801874040143, 5.30380197047756, 5.303802020702321, 5.303802018773521, 5.303802306554287, 5.303802308671905, 5.303802308789084, 5.3038022533568725, 5.303802426607685, 5.3038020174970875, 5.303802258590997, 5.303801816924812, 5.303802252463347, 5.303801817303132, 5.303801911242314, 5.303802270755256, 5.303802270755256, 5.3038014494563726, 5.303802241647157, 5.2955383167433805, 5.3038021733863285, 5.303801306972341, 5.303802194932162, 5.302988517854619, 5.303801321478058, 5.303802543292747, 5.301924577120948, 5.303802697695035, 5.303801152863885, 5.30174127387064, 5.3038031346107655, 5.303802920661738, 5.303803337053113, 5.303802690622186, 5.30380328730827, 5.303802598075633, 5.303803095784797, 5.303802522679819, 5.303803346543543, 5.303378598822103, 5.303397274565091, 5.302969212288431, 5.303803266466117, 5.303802897532757, 5.303803380456019, 5.303802825896973, 5.303803571282035, 5.303802605143625, 5.3022381408726265, 5.303802397701247, 5.303803329777752, 5.303678219489457, 5.30380332954431, 5.303803329967211, 5.3038026928092155, 5.303803274339918, 5.303802447517114, 5.298387309091632, 5.303802442866676, 5.3038034478869065, 5.30380258338441, 5.303803590468577, 5.30347654803991, 5.303802369521527, 5.303802921075391, 5.303802275912222, 5.3038021300347875, 5.303803259610138, 5.303802130075198, 5.303803331597807, 5.30380361213208, 5.303803519420926, 5.303803749093218, 5.303803239393774, 5.303803742473644, 5.303803840021452, 5.30380336590893, 5.303804056643223, 5.303803919342863, 5.303804047245814, 5.297340693419422, 5.30380444756182, 5.29657364073422, 5.303804663603254, 5.303803371460676, 5.303659557292107, 5.303803238969069, 5.303805072670862, 5.3037997357137705, 5.303803091844225, 5.30380583106699, 5.303805829846315, 5.303802899930025]

fig, ax = plt.subplots()
plt.title('Fitness')
plt.xlabel('Generation')
plt.ylabel('Fitness')
ax.plot(range(1000),fitness, 'c')
plt.savefig('./fitnessvgen.png')
plt.show()


